# 4.2 模型参数的访问、初始化和共享

In [1]:
import torch
from torch import nn
from torch.nn import init

print(torch.__version__)

1.4.0


In [12]:
net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1))  # pytorch已进行默认初始化

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


## 4.2.1 访问模型参数

In [4]:
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param)

<class 'generator'>
0.weight Parameter containing:
tensor([[-0.4422,  0.4607,  0.2517, -0.0540],
        [-0.1370,  0.4922,  0.0174, -0.4378],
        [ 0.3644, -0.4144, -0.1559, -0.4753]], requires_grad=True)
0.bias Parameter containing:
tensor([ 0.2040,  0.0126, -0.2825], requires_grad=True)
2.weight Parameter containing:
tensor([[-0.0896, -0.5070,  0.1243]], requires_grad=True)
2.bias Parameter containing:
tensor([-0.3571], requires_grad=True)


In [6]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [7]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)
    def forward(self, x):
        pass
    
n = MyModel()
for name, param in n.named_parameters():
    print(name)

weight1


In [13]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad)
Y.backward()
print(weight_0.grad)

tensor([[-0.2523,  0.1259,  0.3303, -0.0402],
        [-0.0206,  0.1808, -0.1354, -0.2665],
        [-0.2909,  0.3062, -0.0586, -0.3854]])
None
tensor([[ 0.7061,  0.9404,  0.3022,  0.5955],
        [-0.1652, -0.1946, -0.0912, -0.0731],
        [ 0.5193,  0.6917,  0.2223,  0.4380]])


## 4.2.2 初始化模型参数

In [14]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

0.weight tensor([[ 0.0094,  0.0132, -0.0172, -0.0031],
        [ 0.0079,  0.0070, -0.0069, -0.0107],
        [ 0.0024,  0.0120,  0.0059,  0.0049]])
2.weight tensor([[-0.0122, -0.0040, -0.0091]])


In [15]:
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


## 4.2.3 自定义初始化方法

In [16]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()

In [17]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

0.weight tensor([[ 0.0000, -7.7076,  0.0000, -0.0000],
        [-0.0000,  7.0414,  0.0000, -0.0000],
        [ 8.4258,  0.0000, -0.0000,  0.0000]])
2.weight tensor([[-5.3970, -0.0000, -5.5524]])


In [18]:
for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


## 4.2.4 共享模型参数

In [47]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear) 
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [48]:
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

True
True


In [49]:
net.zero_grad()
x = torch.tensor([[1.]])
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad)

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])


### 作为比较，看看不是同一个层的情况：

In [42]:
linear1 = nn.Linear(1, 1, bias=False)
linear2 = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear1, linear2) 
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])
1.weight tensor([[3.]])


In [43]:
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

False
False


In [46]:
net.zero_grad()
x = torch.tensor([[1.]])
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad)

tensor(9., grad_fn=<SumBackward0>)
tensor([[3.]])
